# Treinamento modelo TabNet

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install pytorch-tabnet

## Importações e tratamento inicial

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
import matplotlib.pyplot as plt
import gc
import torch
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report

### Tratamento

In [ ]:
def preencher_valores_com_media_mediana(df):
    for col in df.columns:
        if df[col].dtype in ['float64', 'int64'] and df[col].isnull().sum() > 0:  # Verifica se a coluna é numérica e se tem nulos
            skewness = df[col].skew()  # Calcula a skewness da coluna
            if skewness >= -0.5 and skewness <= 0.5:
                # Distribuição simétrica, preenche com a média
                media = df[col].mean()
                df[col] = df[col].fillna(media)
                # print(f"Coluna '{col}' classificada como simétrica. Preenchendo nulos com a média: {media}")
            else:
                # Distribuição assimétrica, preenche com a mediana
                mediana = df[col].median()
                df[col] = df[col].fillna(mediana)
                # print(f"Coluna '{col}' classificada como assimétrica. Preenchendo nulos com a mediana: {mediana}")
    return df

In [ ]:
# 1. Carregar dados de treino
print("Carregando dados...")
df_train = pd.read_csv('DF_amostra_02.csv')

df_train = df_train.dropna(subset=['y'])

In [ ]:
# 2. Preparar dados
print("\nPreparando dados...")
X = df_train.drop('y', axis=1)
y = df_train['y']

In [ ]:
# 3. Dividir em treino e teste (validação)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
# 4. Tratar valores nulos nos conjuntos
print("\nTratando valores nulos...")
X_train = preencher_valores_com_media_mediana(X_train)
X_val = preencher_valores_com_media_mediana(X_val)


In [ ]:
# 5. Converter para numpy arrays
X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

In [ ]:
# 6. Definir parâmetros e treinar
print("\nConfigurando e treinando modelo...")
model = TabNetClassifier(
    n_d=128,
    n_a=128,
    n_steps=3,
    gamma=1.3,
    n_independent=2,
    n_shared=2,
    lambda_sparse=1e-4,
    optimizer_params={'lr': 0.02},
    verbose=1
)

In [ ]:
# 7. Treinar modelo
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['auc', 'accuracy', 'balanced_accuracy', 'logloss'],
    max_epochs=30,
    patience=10,
    batch_size=512,
    virtual_batch_size=128
)

## Salvar modelo e histórico de parâmetros

In [ ]:

print("\nSalvando o modelo...")
model.save_model('TabNet_v5 128d, 512bs.zip')

In [ ]:
history = model.history
# pd.DataFrame(history).to_csv('training_history.csv', index=False)
print(history)

In [ ]:
history_dict = {
    'epoch': range(len(model.history['loss'])),
    'loss': model.history['loss'],
    'train_auc': model.history['train_auc'],
    'valid_auc': model.history['valid_auc']
}
history_df = pd.DataFrame(history_dict)
history_df.to_csv('training_history.csv', index=False)

### Métricas do modelo durante treinamento

In [ ]:
# Plotar métricas ao longo do treinamento / Verificar overfitting AUC-ROC
plt.figure(figsize=(15, 5))

# Plot Loss
plt.subplot(1, 3, 1)
plt.plot(model.history['loss'])
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

# Plot AUC
plt.subplot(1, 3, 2)
plt.plot(model.history['train_auc'], label='Train')
plt.plot(model.history['valid_auc'], label='Valid')
plt.title('AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Validação do modelo no dataset de test real

In [ ]:
def test_model(pathfile):
    df_test = pd.read_csv(pathfile + '.csv')
    # Separar features e target
    X_test = df_test.drop('y', axis=1)
    y_test = df_test['y']  # Pegando os valores reais

    # Preparar dados
    X_test = preencher_valores_com_media_mediana(X_test)
    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    return X_test, y_test

In [ ]:
X_test, y_test = test_model('test')

# 8. Fazer predições no conjunto de teste
print("\nFazendo predições no conjunto de teste...")
test_predictions = model.predict_proba(X_test)

prediction = model.predict(X_test)

# Calculando e mostrando a acurácia
acuracia = accuracy_score(y_test, prediction)
print(f"\nAcurácia do modelo: {acuracia:.4f}")

# Mostrando relatório detalhado de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, prediction))

# Verificando e mostrando a importância das features
importancia_features = pd.DataFrame({
    'feature': X.columns,  # Mantemos as colunas originais do DataFrame
    'importancia': model.feature_importances_
})
print("\nImportância das Features:")
print(importancia_features.sort_values('importancia', ascending=False))

In [ ]:
# Calcular ROC AUC
fpr, tpr, _ = roc_curve(y_test, test_predictions[:, 1])
auc = roc_auc_score(y_test, test_predictions[:, 1])

In [ ]:
# Plot da curva ROC
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')  # linha diagonal
plt.title('Curva ROC no conjunto de teste')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.legend()
plt.grid(True)


In [ ]:
prediction = model.predict(X_test)

# Calculando e mostrando a acurácia
acuracia = accuracy_score(y_test, prediction)
print(f"\nAcurácia do modelo: {acuracia:.4f}")

# Mostrando relatório detalhado de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, prediction))

# Verificando e mostrando a importância das features
importancia_features = pd.DataFrame({
    'feature': X.columns,  # Mantemos as colunas originais do DataFrame
    'importancia': model.feature_importances_
})
print("\nImportância das Features:")
print(importancia_features.sort_values('importancia', ascending=False))